## Model Training

1.1 Import Data and Required Packages 

In [32]:
#basic import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Modeling
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings

Import the CSV Data as Pandas DataFrame

In [19]:
df = pd.read_csv('data/raw.csv')

In [20]:
df.head(5)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Preparaing X and y variales

In [21]:
X = df.drop(columns=['math_score'],axis=1)

In [22]:
X.shape

(1003, 7)

In [23]:
y = df.math_score

In [9]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [24]:
y

0       72
1       69
2       90
3       47
4       76
        ..
998     68
999     77
1000    77
1001    77
1002    77
Name: math_score, Length: 1003, dtype: int64

In [25]:
#create column transformer with 3 types transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer([
    ("OneHotEncoder", oh_transformer, cat_features),
    ("StandardScaler",numeric_transformer,num_features)
])

In [26]:
X = preprocessor.fit_transform(X)

In [27]:
X.shape

(1003, 19)

In [30]:
#seperate data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((802, 19), (201, 19))

Create an Evaluate function to give all metrics after model Training

In [40]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, rmse,r2_square

In [46]:
models = {

    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "RandomForestRegressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBostRegressor" : CatBoostRegressor(),
    "AdaBoost Regressor" : AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    #make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluate train and test dataset

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)

    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


    


LinearRegression
Model performance for Training set
- Root Mean Squared Error: 5.2983
- Mean Absolute Error: 4.2472
- R2 Score: 0.8711
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4248
- Mean Absolute Error: 4.2022
- R2 Score: 0.8925


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5625
- Mean Absolute Error: 5.1941
- R2 Score: 0.8022
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5845
- Mean Absolute Error: 5.1295
- R2 Score: 0.8416


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.2967
- Mean Absolute Error: 4.2525
- R2 Score: 0.8712
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4184
- Mean Absolute Error: 4.1972
- R2 Score: 0.8927


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.7917
- Mean Absolute Error: 4.5731
- R2 Score: 0.8460
------------------------

d:\MLProject\env\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


XGBRegressor
Model performance for Training set
- Root Mean Squared Error: 0.9877
- Mean Absolute Error: 0.6211
- R2 Score: 0.9955
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.9126
- Mean Absolute Error: 5.3333
- R2 Score: 0.8254


Learning rate set to 0.03954
0:	learn: 14.3487910	total: 207ms	remaining: 3m 26s
1:	learn: 14.0060245	total: 212ms	remaining: 1m 46s
2:	learn: 13.6672288	total: 214ms	remaining: 1m 11s
3:	learn: 13.3099539	total: 217ms	remaining: 54.1s
4:	learn: 12.9774304	total: 220ms	remaining: 43.7s
5:	learn: 12.6652412	total: 222ms	remaining: 36.8s
6:	learn: 12.3613944	total: 225ms	remaining: 32s
7:	learn: 12.1015901	total: 227ms	remaining: 28.2s
8:	learn: 11.8191092	total: 230ms	remaining: 25.3s
9:	learn: 11.5339425	total: 232ms	remaining: 23s
10:	learn: 11.2524955	total: 234ms	remaining: 21.1s
11:	learn: 11.0007982	total: 238ms	remaining: 19.6s
12:	learn: 10.7694114	total: 240ms	remaining: 18.2s
13:	learn: 10.5286965	to

In [37]:
len(list(models.keys))

9

In [43]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'K-Neighbors Regressor', 'Decision Tree', 'RandomForestRegressor', 'XGBRegressor', 'CatBostRegressor', 'AdaBoost Regressor'])

### Results

In [47]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.892724
0,LinearRegression,0.892470
7,CatBostRegressor,0.860638
5,RandomForestRegressor,0.848783
1,Lasso,0.841583
8,AdaBoost Regressor,0.831031
6,XGBRegressor,0.825401
3,K-Neighbors Regressor,0.809244
4,Decision Tree,0.745064
